# Jogeset Hálózat Elemzése
Ebben a notebookban a `graph_builder.py` szkript által létrehozott jogeset hálózatot elemezzük.

In [ ]:
# Szükséges könyvtárak importálása
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import json
import os
import sys

# Projekt gyökérkönyvtár hozzáadása a path-hoz a config betöltéséhez
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

try:
    from configs import config
    GRAPH_PATH_GML = config.GRAPH_OUTPUT_GML_PATH
    GRAPH_PATH_JSON = config.GRAPH_OUTPUT_JSON_PATH
    METADATA_PATH = config.GRAPH_METADATA_PATH
except ImportError:
    print('Config file not found, using default paths.')
    GRAPH_PATH_GML = '../processed_data/output_graph.graphml'
    GRAPH_PATH_JSON = '../processed_data/output_graph.json'
    METADATA_PATH = '../processed_data/output_graph_metadata.json'

# Matplotlib beállítások (opcionális)
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

## Gráf Betöltése
Betöltjük a gráfot a GML fájlból. A GML formátum általában jobban megőrzi az attribútumokat, mint a JSON node-link formátum.

In [ ]:
try:
    G = nx.read_graphml(GRAPH_PATH_GML)
    print(f'Gráf sikeresen betöltve: {GRAPH_PATH_GML}')
    print(f'Csomópontok száma: {G.number_of_nodes()}')
    print(f'Élek száma: {G.number_of_edges()}')
except FileNotFoundError:
    print(f'Hiba: A GML fájl nem található: {GRAPH_PATH_GML}')
    G = None # Gráf inicializálása None-ra, ha a betöltés sikertelen
except Exception as e:
    print(f'Hiba a GML fájl betöltése közben: {e}')
    G = None

## Metaadatok Betöltése

In [ ]:
try:
    with open(METADATA_PATH, 'r', encoding='utf-8') as f:
        metadata = json.load(f)
    print('Metaadatok:')
    for key, value in metadata.items():
        print(f'- {key}: {value}')
except FileNotFoundError:
    print(f'Hiba: A metaadat fájl nem található: {METADATA_PATH}')
    metadata = None
except Exception as e:
    print(f'Hiba a metaadat fájl betöltése közben: {e}')
    metadata = None

## Alapvető Gráf Jellemzők
Kiírjuk a gráf alapvető jellemzőit, mint a csomópontok és élek száma, valamint az átlagos fokszám.

In [ ]:
if G is not None:
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    
    if num_nodes > 0:
        # Fokszámok kiszámítása (be-, ki- és teljes fokszám irányított gráf esetén)
        in_degrees = dict(G.in_degree())
        out_degrees = dict(G.out_degree())
        total_degrees = dict(G.degree())
        
        avg_in_degree = sum(in_degrees.values()) / num_nodes
        avg_out_degree = sum(out_degrees.values()) / num_nodes
        avg_total_degree = sum(total_degrees.values()) / num_nodes
        
        print(f'Átlagos bejövő fokszám: {avg_in_degree:.2f}')
        print(f'Átlagos kimenő fokszám: {avg_out_degree:.2f}')
        print(f'Átlagos teljes fokszám: {avg_total_degree:.2f}')
    else:
        print('A gráf üres, nincs csomópont.')
        
    # Gráf sűrűsége
    density = nx.density(G)
    print(f'Gráf sűrűsége: {density:.6f}')
else:
    print('A gráf nincs betöltve.')

## Csomópont Típusok Elemzése
Megszámoljuk a különböző típusú csomópontokat (pl. dokumentum, bíróság, jogszabály).

In [ ]:
if G is not None:
    node_types = nx.get_node_attributes(G, 'type')
    type_counts = pd.Series(node_types).value_counts()
    
    print('Csomópont típusok eloszlása:')
    print(type_counts)
    
    # Oszlopdiagram a típusokról
    fig = px.bar(type_counts, x=type_counts.index, y=type_counts.values, labels={'x':'Típus', 'y':'Darabszám'}, title='Csomópont Típusok Eloszlása')
    fig.show()
else:
    print('A gráf nincs betöltve.')

## Fokszám Eloszlás Vizualizációja
Megvizsgáljuk a bejövő, kimenő és teljes fokszámok eloszlását.

In [ ]:
if G is not None and num_nodes > 0:
    # Adatkeretek létrehozása a fokszámokból
    in_degree_df = pd.DataFrame(in_degrees.items(), columns=['node', 'in_degree']).sort_values('in_degree', ascending=False)
    out_degree_df = pd.DataFrame(out_degrees.items(), columns=['node', 'out_degree']).sort_values('out_degree', ascending=False)
    total_degree_df = pd.DataFrame(total_degrees.items(), columns=['node', 'total_degree']).sort_values('total_degree', ascending=False)
    
    print('
Legmagasabb bejövő fokszámú csomópontok:')
    print(in_degree_df.head())
    
    print('
Legmagasabb kimenő fokszámú csomópontok:')
    print(out_degree_df.head())
    
    print('
Legmagasabb teljes fokszámú csomópontok:')
    print(total_degree_df.head())
    
    # Hisztogramok
    plt.figure(figsize=(18, 5))
    
    plt.subplot(1, 3, 1)
    plt.hist(in_degrees.values(), bins=range(min(in_degrees.values()), max(in_degrees.values()) + 1, 1), alpha=0.7)
    plt.title('Bejövő Fokszám Eloszlás')
    plt.xlabel('Bejövő Fokszám')
    plt.ylabel('Csomópontok Száma')
    plt.yscale('log') # Logaritmikus skála hasznos lehet
    
    plt.subplot(1, 3, 2)
    plt.hist(out_degrees.values(), bins=range(min(out_degrees.values()), max(out_degrees.values()) + 1, 1), alpha=0.7, color='orange')
    plt.title('Kimenő Fokszám Eloszlás')
    plt.xlabel('Kimenő Fokszám')
    plt.ylabel('Csomópontok Száma')
    plt.yscale('log')
    
    plt.subplot(1, 3, 3)
    plt.hist(total_degrees.values(), bins=range(min(total_degrees.values()), max(total_degrees.values()) + 1, 1), alpha=0.7, color='green')
    plt.title('Teljes Fokszám Eloszlás')
    plt.xlabel('Teljes Fokszám')
    plt.ylabel('Csomópontok Száma')
    plt.yscale('log')
    
    plt.tight_layout()
    plt.show()
else:
    print('A gráf nincs betöltve vagy üres.')

## Gráf Vizualizáció (Minta)
Mivel a teljes gráf valószínűleg túl nagy a közvetlen vizualizációhoz, itt egy kisebb algráfot vagy egy mintát jelenítünk meg. Például a legmagasabb fokszámú csomópontok környezetét.
**Figyelem:** Nagy gráfok esetén ez a lépés lassú lehet és a kimenet átláthatatlan.

In [ ]:
if G is not None and num_nodes > 0:
    # Kiválasztunk néhány központi csomópontot (pl. a legmagasabb fokszámúak)
    central_nodes = total_degree_df['node'].head(5).tolist() # Top 5 csomópont
    
    # Létrehozzuk az ego gráfokat ezekre a csomópontokra (csomópont + közvetlen szomszédok)
    subgraph_nodes = set(central_nodes)
    for node in central_nodes:
        subgraph_nodes.update(G.predecessors(node))
        subgraph_nodes.update(G.successors(node))
        
    # Csak a dokumentum típusú csomópontokat tartjuk meg a szomszédok közül (opcionális szűrés)
    # subgraph_nodes = {n for n in subgraph_nodes if G.nodes[n].get('type') == 'dokumentum' or n in central_nodes}
    
    sub_G = G.subgraph(subgraph_nodes)
    
    print(f'Algráf létrehozva {sub_G.number_of_nodes()} csomóponttal és {sub_G.number_of_edges()} éllel.')

    if sub_G.number_of_nodes() > 0 and sub_G.number_of_nodes() < 500: # Csak ésszerű méretű algráfot rajzolunk ki
        plt.figure(figsize=(15, 15))
        pos = nx.spring_layout(sub_G, k=0.3, iterations=50) # Elrendezés
        
        # Csomópontok színe típus szerint
        node_colors = []
        color_map = {'dokumentum': 'blue', 'jogszabaly': 'red', 'birosag': 'green'}
        for node in sub_G.nodes():
            node_type = G.nodes[node].get('type', 'unknown')
            node_colors.append(color_map.get(node_type, 'grey'))
            
        nx.draw(sub_G, pos, node_color=node_colors, with_labels=False, node_size=50, alpha=0.7, edge_color='gray')
        
        # Címkék csak a központi csomópontokhoz (opcionális, olvashatóság miatt)
        # labels = {n: n for n in central_nodes if n in sub_G}
        # nx.draw_networkx_labels(sub_G, pos, labels=labels, font_size=8)
        
        plt.title('Algráf Vizualizáció (Központi Csomópontok Környezete)')
        plt.show()
    elif sub_G.number_of_nodes() >= 500:
        print('Az algráf túl nagy a vizualizációhoz.')
    else:
        print('Nem sikerült algráfot létrehozni.')
else:
    print('A gráf nincs betöltve vagy üres.')

## További Elemzési Lehetőségek
*   **Centralitás mértékek:** PageRank, Betweenness Centrality, Eigenvector Centrality kiszámítása a legbefolyásosabb csomópontok azonosítására.
*   **Komponens analízis:** Erősen és gyengén összefüggő komponensek vizsgálata.
*   **Közösség detektálás:** Algoritmusok (pl. Louvain) futtatása a gráfban lévő közösségek (klaszterek) feltárására.
*   **Időbeli elemzés:** Ha az adatok tartalmaznak időbélyeget, az időbeli változások vizsgálata.

## Centralitás Mértékek
A centralitás mértékek a gráf elemzés kulcsfontosságú eszközei, amelyek segítenek azonosítani a hálózatban a legbefolyásosabb csomópontokat különböző szempontok szerint. Az alábbiakban kiszámítjuk és összehasonlítjuk a legfontosabb centralitási mértékeket.